# How to save btc safely.

## BIP0039 生成确定性密钥的助记词

<https://en.bitcoin.it/wiki/BIP_0039>

**BTC私钥** 实际上就是一个32字节的随机数组，而 **公钥** 是通过将椭圆曲线的 **生成元点** 自加而得到，自加的次数就是这个私钥 ，此时私钥被当作一个大整数。而 **BTC地址** 则是通过把公钥哈希而得到。程序逻辑如下：

例子来自 <https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses>

In [8]:
import random

# 设置随机数种子，以便结果可重复
random.seed(0)

# 随机生成一个私钥
private_key = random.randint(0, 2**256 - 1)

# 为了方便，我们直接使用一个固定的私钥
private_key = 0x18e14a7b6a307f426a94f8114701e7c8e774e7f9a47e2c2035db29a206321725
print(f"private_key: 0x{private_key:064x}")

private_key: 0x18e14a7b6a307f426a94f8114701e7c8e774e7f9a47e2c2035db29a206321725


In [9]:
# BTC所选择的椭圆曲线是secp256k1, 其参数如下
secp256k1_p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
secp256k1_a = 0
secp256k1_b = 7
secp256k1_n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
secp256k1_Gx = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
secp256k1_Gy = 0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8

# 椭圆曲线的方程是 y^2 = x^3 + ax + b

# 椭圆曲线上的加法如下，其中 None 表示无穷远点，即加法的单位元，即 0，p1和p2是椭圆曲线上的点
def secp256k1_add(p1, p2):
    if p1 is None:
        return p2
    if p2 is None:
        return p1
    if p1[0] == p2[0] and p1[1] != p2[1]:
        # 两个点互为相反数，即相加为0
        assert (p1[1] + p2[1] % secp256k1_p) == 0
        return None
    # s是斜率
    if p1[0] == p2[0]:
        s = (3 * p1[0] * p1[0] + secp256k1_a) * pow(2 * p1[1], -1, secp256k1_p)
    else:
        s = (p2[1] - p1[1]) * pow(p2[0] - p1[0], -1, secp256k1_p)
    x = (s * s - p1[0] - p2[0]) % secp256k1_p
    y = ( - (s * (x - p1[0]) + p1[1]) ) % secp256k1_p
    return (x, y)

# 私钥 -> 公钥 (倍点算法)
def priv_key_to_pub_key(priv_key):
    assert priv_key < 2**256
    # 生成公钥
    pub_key = None
    base = (secp256k1_Gx, secp256k1_Gy)
    for i in range(256):
        if priv_key & (1 << i):
            pub_key = secp256k1_add(pub_key, base)
        base = secp256k1_add(base, base)
    return pub_key

# 生成公钥
pub_key = priv_key_to_pub_key(private_key)
print(f"pub_key: 0x{pub_key[0]:064x}, 0x{pub_key[1]:064x}")

# 压缩形式的公钥
if pub_key[1] & 1:
    compressed_pub_key = 0x03
else:
    compressed_pub_key = 0x02
compressed_pub_key = compressed_pub_key << 256 | pub_key[0]
print(f"compressed_pub_key: 0x{compressed_pub_key:066x}")


pub_key: 0x50863ad64a87ae8a2fe83c1af1a8403cb53f53e486d8511dad8a04887e5b2352, 0x2cd470243453a299fa9e77237716103abc11a1df38855ed6f2ee187e9c582ba6
compressed_pub_key: 0x0250863ad64a87ae8a2fe83c1af1a8403cb53f53e486d8511dad8a04887e5b2352


In [10]:
import hashlib

# 生成地址
hashed_pub_key = hashlib.sha256(compressed_pub_key.to_bytes(33, 'big')).digest()
print(f"sha256ed_pub_key: 0x{int.from_bytes(hashed_pub_key, 'big'):064x}")

ripemd160_hasher = hashlib.new('ripemd160')
ripemd160_hasher.update(hashed_pub_key)
ripemd160_pub_key = ripemd160_hasher.digest()
print(f"ripemd160ed_pub_key: 0x{int.from_bytes(ripemd160_pub_key, 'big'):040x}")

ripemd160_pub_key_with_version = int.from_bytes(ripemd160_pub_key, 'big').to_bytes(21, 'big') # 添加版本号
print(f"ripemd160_pub_key_with_version: 0x{int.from_bytes(ripemd160_pub_key_with_version, 'big'):042x}")

hashed_ripemd160_pub_key_with_version = hashlib.sha256(ripemd160_pub_key_with_version).digest()
print(f"sha256ed_ripemd160_pub_key_with_version: 0x{int.from_bytes(hashed_ripemd160_pub_key_with_version, 'big'):064x}")

hashed_twice_ripemd160_pub_key_with_version = hashlib.sha256(hashed_ripemd160_pub_key_with_version).digest()
print(f"sha256ed_twice_ripemd160_pub_key_with_version: 0x{int.from_bytes(hashed_twice_ripemd160_pub_key_with_version, 'big'):064x}")

checksum = hashed_twice_ripemd160_pub_key_with_version[:4]
print(f"checksum: 0x{int.from_bytes(checksum, 'big'):08x}")

address = ripemd160_pub_key_with_version + checksum
print(f"address: 0x{int.from_bytes(address, 'big'):050x}")

# base58编码
import base58
address = base58.b58encode(address)
print(f"base58 address: {address.decode('utf-8')}")
assert address.decode('utf-8') == "1PMycacnJaSqwwJqjawXBErnLsZ7RkXUAs"

sha256ed_pub_key: 0x0b7c28c9b7290c98d7438e70b3d3f7c848fbd7d1dc194ff83f4f7cc9b1378e98
ripemd160ed_pub_key: 0xf54a5851e9372b87810a8e60cdd2e7cfd80b6e31
ripemd160_pub_key_with_version: 0x00f54a5851e9372b87810a8e60cdd2e7cfd80b6e31
sha256ed_ripemd160_pub_key_with_version: 0xad3c854da227c7e99c4abfad4ea41d71311160df2e415e713318c70d67c6b41c
sha256ed_twice_ripemd160_pub_key_with_version: 0xc7f18fe8fcbed6396741e58ad259b5cb16b7fd7f041904147ba1dcffabf747fd
checksum: 0xc7f18fe8
address: 0x00f54a5851e9372b87810a8e60cdd2e7cfd80b6e31c7f18fe8
base58 address: 1PMycacnJaSqwwJqjawXBErnLsZ7RkXUAs


上面就是生成私钥，以及从私钥生成比特币地址的完整过程。为了防止币的丢失，就必须保证私钥被妥善保管。私钥的随机性，导致了私钥难于准确的被记录。BIP0039尝试解决这个问题。

BIP0039中，我们首先生成 **随机比特** ，然后根据随机比特生成 **助记词** ，然后根据助记词生成 **私钥**。我们只需要记住助记词，就能随时重新生成私钥，从而生成公钥以及地址。而助记词由若干单词组成，方便记忆或者记录。为了进一步方便记忆和记录，BIP0039对助记词的选择做了限制，并且还添加了额外的冗余防止记忆出错。

助记词词典由2048个英文单词组成。这个词典还有一些特殊性质，一切都是为了更容易记忆或者记录。

1. 只要前4个字母，就能区分所有单词。
2. 相似的词语不选用。
3. 单词列表是有序的。

完整的单词列表如下：

In [11]:
mnemonic_word_list = 'abandon ability able about above absent absorb abstract absurd abuse access accident account accuse achieve acid acoustic acquire across act action actor actress actual adapt add addict address adjust admit adult advance advice aerobic affair afford afraid again age agent agree ahead aim air airport aisle alarm album alcohol alert alien all alley allow almost alone alpha already also alter always amateur amazing among amount amused analyst anchor ancient anger angle angry animal ankle announce annual another answer antenna antique anxiety any apart apology appear apple approve april arch arctic area arena argue arm armed armor army around arrange arrest arrive arrow art artefact artist artwork ask aspect assault asset assist assume asthma athlete atom attack attend attitude attract auction audit august aunt author auto autumn average avocado avoid awake aware away awesome awful awkward axis baby bachelor bacon badge bag balance balcony ball bamboo banana banner bar barely bargain barrel base basic basket battle beach bean beauty because become beef before begin behave behind believe below belt bench benefit best betray better between beyond bicycle bid bike bind biology bird birth bitter black blade blame blanket blast bleak bless blind blood blossom blouse blue blur blush board boat body boil bomb bone bonus book boost border boring borrow boss bottom bounce box boy bracket brain brand brass brave bread breeze brick bridge brief bright bring brisk broccoli broken bronze broom brother brown brush bubble buddy budget buffalo build bulb bulk bullet bundle bunker burden burger burst bus business busy butter buyer buzz cabbage cabin cable cactus cage cake call calm camera camp can canal cancel candy cannon canoe canvas canyon capable capital captain car carbon card cargo carpet carry cart case cash casino castle casual cat catalog catch category cattle caught cause caution cave ceiling celery cement census century cereal certain chair chalk champion change chaos chapter charge chase chat cheap check cheese chef cherry chest chicken chief child chimney choice choose chronic chuckle chunk churn cigar cinnamon circle citizen city civil claim clap clarify claw clay clean clerk clever click client cliff climb clinic clip clock clog close cloth cloud clown club clump cluster clutch coach coast coconut code coffee coil coin collect color column combine come comfort comic common company concert conduct confirm congress connect consider control convince cook cool copper copy coral core corn correct cost cotton couch country couple course cousin cover coyote crack cradle craft cram crane crash crater crawl crazy cream credit creek crew cricket crime crisp critic crop cross crouch crowd crucial cruel cruise crumble crunch crush cry crystal cube culture cup cupboard curious current curtain curve cushion custom cute cycle dad damage damp dance danger daring dash daughter dawn day deal debate debris decade december decide decline decorate decrease deer defense define defy degree delay deliver demand demise denial dentist deny depart depend deposit depth deputy derive describe desert design desk despair destroy detail detect develop device devote diagram dial diamond diary dice diesel diet differ digital dignity dilemma dinner dinosaur direct dirt disagree discover disease dish dismiss disorder display distance divert divide divorce dizzy doctor document dog doll dolphin domain donate donkey donor door dose double dove draft dragon drama drastic draw dream dress drift drill drink drip drive drop drum dry duck dumb dune during dust dutch duty dwarf dynamic eager eagle early earn earth easily east easy echo ecology economy edge edit educate effort egg eight either elbow elder electric elegant element elephant elevator elite else embark embody embrace emerge emotion employ empower empty enable enact end endless endorse enemy energy enforce engage engine enhance enjoy enlist enough enrich enroll ensure enter entire entry envelope episode equal equip era erase erode erosion error erupt escape essay essence estate eternal ethics evidence evil evoke evolve exact example excess exchange excite exclude excuse execute exercise exhaust exhibit exile exist exit exotic expand expect expire explain expose express extend extra eye eyebrow fabric face faculty fade faint faith fall false fame family famous fan fancy fantasy farm fashion fat fatal father fatigue fault favorite feature february federal fee feed feel female fence festival fetch fever few fiber fiction field figure file film filter final find fine finger finish fire firm first fiscal fish fit fitness fix flag flame flash flat flavor flee flight flip float flock floor flower fluid flush fly foam focus fog foil fold follow food foot force forest forget fork fortune forum forward fossil foster found fox fragile frame frequent fresh friend fringe frog front frost frown frozen fruit fuel fun funny furnace fury future gadget gain galaxy gallery game gap garage garbage garden garlic garment gas gasp gate gather gauge gaze general genius genre gentle genuine gesture ghost giant gift giggle ginger giraffe girl give glad glance glare glass glide glimpse globe gloom glory glove glow glue goat goddess gold good goose gorilla gospel gossip govern gown grab grace grain grant grape grass gravity great green grid grief grit grocery group grow grunt guard guess guide guilt guitar gun gym habit hair half hammer hamster hand happy harbor hard harsh harvest hat have hawk hazard head health heart heavy hedgehog height hello helmet help hen hero hidden high hill hint hip hire history hobby hockey hold hole holiday hollow home honey hood hope horn horror horse hospital host hotel hour hover hub huge human humble humor hundred hungry hunt hurdle hurry hurt husband hybrid ice icon idea identify idle ignore ill illegal illness image imitate immense immune impact impose improve impulse inch include income increase index indicate indoor industry infant inflict inform inhale inherit initial inject injury inmate inner innocent input inquiry insane insect inside inspire install intact interest into invest invite involve iron island isolate issue item ivory jacket jaguar jar jazz jealous jeans jelly jewel job join joke journey joy judge juice jump jungle junior junk just kangaroo keen keep ketchup key kick kid kidney kind kingdom kiss kit kitchen kite kitten kiwi knee knife knock know lab label labor ladder lady lake lamp language laptop large later latin laugh laundry lava law lawn lawsuit layer lazy leader leaf learn leave lecture left leg legal legend leisure lemon lend length lens leopard lesson letter level liar liberty library license life lift light like limb limit link lion liquid list little live lizard load loan lobster local lock logic lonely long loop lottery loud lounge love loyal lucky luggage lumber lunar lunch luxury lyrics machine mad magic magnet maid mail main major make mammal man manage mandate mango mansion manual maple marble march margin marine market marriage mask mass master match material math matrix matter maximum maze meadow mean measure meat mechanic medal media melody melt member memory mention menu mercy merge merit merry mesh message metal method middle midnight milk million mimic mind minimum minor minute miracle mirror misery miss mistake mix mixed mixture mobile model modify mom moment monitor monkey monster month moon moral more morning mosquito mother motion motor mountain mouse move movie much muffin mule multiply muscle museum mushroom music must mutual myself mystery myth naive name napkin narrow nasty nation nature near neck need negative neglect neither nephew nerve nest net network neutral never news next nice night noble noise nominee noodle normal north nose notable note nothing notice novel now nuclear number nurse nut oak obey object oblige obscure observe obtain obvious occur ocean october odor off offer office often oil okay old olive olympic omit once one onion online only open opera opinion oppose option orange orbit orchard order ordinary organ orient original orphan ostrich other outdoor outer output outside oval oven over own owner oxygen oyster ozone pact paddle page pair palace palm panda panel panic panther paper parade parent park parrot party pass patch path patient patrol pattern pause pave payment peace peanut pear peasant pelican pen penalty pencil people pepper perfect permit person pet phone photo phrase physical piano picnic picture piece pig pigeon pill pilot pink pioneer pipe pistol pitch pizza place planet plastic plate play please pledge pluck plug plunge poem poet point polar pole police pond pony pool popular portion position possible post potato pottery poverty powder power practice praise predict prefer prepare present pretty prevent price pride primary print priority prison private prize problem process produce profit program project promote proof property prosper protect proud provide public pudding pull pulp pulse pumpkin punch pupil puppy purchase purity purpose purse push put puzzle pyramid quality quantum quarter question quick quit quiz quote rabbit raccoon race rack radar radio rail rain raise rally ramp ranch random range rapid rare rate rather raven raw razor ready real reason rebel rebuild recall receive recipe record recycle reduce reflect reform refuse region regret regular reject relax release relief rely remain remember remind remove render renew rent reopen repair repeat replace report require rescue resemble resist resource response result retire retreat return reunion reveal review reward rhythm rib ribbon rice rich ride ridge rifle right rigid ring riot ripple risk ritual rival river road roast robot robust rocket romance roof rookie room rose rotate rough round route royal rubber rude rug rule run runway rural sad saddle sadness safe sail salad salmon salon salt salute same sample sand satisfy satoshi sauce sausage save say scale scan scare scatter scene scheme school science scissors scorpion scout scrap screen script scrub sea search season seat second secret section security seed seek segment select sell seminar senior sense sentence series service session settle setup seven shadow shaft shallow share shed shell sheriff shield shift shine ship shiver shock shoe shoot shop short shoulder shove shrimp shrug shuffle shy sibling sick side siege sight sign silent silk silly silver similar simple since sing siren sister situate six size skate sketch ski skill skin skirt skull slab slam sleep slender slice slide slight slim slogan slot slow slush small smart smile smoke smooth snack snake snap sniff snow soap soccer social sock soda soft solar soldier solid solution solve someone song soon sorry sort soul sound soup source south space spare spatial spawn speak special speed spell spend sphere spice spider spike spin spirit split spoil sponsor spoon sport spot spray spread spring spy square squeeze squirrel stable stadium staff stage stairs stamp stand start state stay steak steel stem step stereo stick still sting stock stomach stone stool story stove strategy street strike strong struggle student stuff stumble style subject submit subway success such sudden suffer sugar suggest suit summer sun sunny sunset super supply supreme sure surface surge surprise surround survey suspect sustain swallow swamp swap swarm swear sweet swift swim swing switch sword symbol symptom syrup system table tackle tag tail talent talk tank tape target task taste tattoo taxi teach team tell ten tenant tennis tent term test text thank that theme then theory there they thing this thought three thrive throw thumb thunder ticket tide tiger tilt timber time tiny tip tired tissue title toast tobacco today toddler toe together toilet token tomato tomorrow tone tongue tonight tool tooth top topic topple torch tornado tortoise toss total tourist toward tower town toy track trade traffic tragic train transfer trap trash travel tray treat tree trend trial tribe trick trigger trim trip trophy trouble truck true truly trumpet trust truth try tube tuition tumble tuna tunnel turkey turn turtle twelve twenty twice twin twist two type typical ugly umbrella unable unaware uncle uncover under undo unfair unfold unhappy uniform unique unit universe unknown unlock until unusual unveil update upgrade uphold upon upper upset urban urge usage use used useful useless usual utility vacant vacuum vague valid valley valve van vanish vapor various vast vault vehicle velvet vendor venture venue verb verify version very vessel veteran viable vibrant vicious victory video view village vintage violin virtual virus visa visit visual vital vivid vocal voice void volcano volume vote voyage wage wagon wait walk wall walnut want warfare warm warrior wash wasp waste water wave way wealth weapon wear weasel weather web wedding weekend weird welcome west wet whale what wheat wheel when where whip whisper wide width wife wild will win window wine wing wink winner winter wire wisdom wise wish witness wolf woman wonder wood wool word work world worry worth wrap wreck wrestle wrist write wrong yard year yellow you young youth zebra zero zone zoo'
mnemonic_word_list = mnemonic_word_list.split(' ')

assert len(mnemonic_word_list) == 2048

# 检查前4位是否能唯一确定一个助记词
def check_mnemonic_word_list(mnemonic_word_list):
    mnemonic_word_list = list(mnemonic_word_list)
    mnemonic_word_list = [word[:4] for word in mnemonic_word_list]
    mnemonic_word_list = set(mnemonic_word_list)
    assert len(mnemonic_word_list) == 2048
check_mnemonic_word_list(mnemonic_word_list)

print(f"mnemonic_word_list: {','.join(mnemonic_word_list[0:4])},...")

mnemonic_word_list: abandon,ability,able,about,...


助记词的个数可以是12个或者更多，我们以12个助记词为例来说明BIP0039。

12个助记词，每个助记词从2048个单词中随机挑选，说明我们至少需要 `12 * 11 = 132` 个随机比特。在BIP0039中，12个助记词我们使用4个比特的校验码，因此剩余的128个比特来自随机。示例如下：

测试例子来自 <https://github.com/trezor/python-mnemonic/blob/master/vectors.json>

In [12]:
random_bits = random.randint(0, 2**128 - 1)

# 为了方便，我们直接使用一个固定的随机数
random_bits = 0x9e885d952ad362caeb4efe34a8e91bd2

random_bytes = random_bits.to_bytes(16, 'big')
sha256ed_random_bytes = hashlib.sha256(random_bytes).digest()
sha256ed_random_bytes_as_int = int.from_bytes(sha256ed_random_bytes, 'big')
random_bits = (random_bits << 4) | (sha256ed_random_bytes_as_int >> (256 - 4))

mnemonic = []
for i in range(12):
    mnemonic.append(mnemonic_word_list[random_bits & 2047])
    random_bits = random_bits >> 11
mnemonic = mnemonic[::-1]
mnemonic = ' '.join(mnemonic)
print(f"mnemonic: {mnemonic}")

mnemonic: ozone drill grab fiber curtain grace pudding thank cruise elder eight picnic


In [13]:
# 这里选择TREZOR作为密码，是因为我们的测试例子来自TREZOR
salt = "mnemonic" + "TREZOR"

# password based key derivation function 2
seed = hashlib.pbkdf2_hmac('sha512', mnemonic.encode('utf-8'), salt.encode('utf-8'), 2048)
print(f"seed: 0x{int.from_bytes(seed, 'big'):0128x}")

def my_hmac(key_bytes, msg_bytes):
    block_size_of_sha512 = 128
    if len(key_bytes) > block_size_of_sha512:
        key_bytes = hashlib.sha512(key_bytes).digest() + bytes(block_size_of_sha512 - 64)
    else:
        key_bytes = key_bytes + bytes(block_size_of_sha512 - len(key_bytes))
    opad = 0x5c
    ipad = 0x36

    part1 = bytes([x ^ opad for x in key_bytes])
    part2 = bytes([x ^ ipad for x in key_bytes]) + msg_bytes
    return hashlib.sha512(part1 + hashlib.sha512(part2).digest()).digest()

assert int.from_bytes(my_hmac( "key".encode('utf-8'), "The quick brown fox jumps over the lazy dog".encode('utf-8') ), "big") == \
        0xb42af09057bac1e2d41708e48a902e09b5ff7f12ab428a4fe86653c73dd248fb82f948a549f7b791a5b41915ee4d1ec3935357e4e2317250d0372afa2ebeeb3a

def my_pbkdf2(mnemonic_bytes, salt_bytes):
    u = salt_bytes + (1).to_bytes(4, 'big')
    ret = None
    for _ in range(2048):
        u = my_hmac(mnemonic_bytes, u)
        if ret is None:
            ret = u
        else:
            ret = bytes([x ^ y for x, y in zip(ret, u)])
    return ret

seed2 = my_pbkdf2(mnemonic.encode('utf-8'), salt.encode('utf-8'))
assert int.from_bytes(seed, 'big') == int.from_bytes(seed2, 'big')


seed: 0x274ddc525802f7c828d8ef7ddbcdc5304e87ac3535913611fbbfa986d0c9e5476c91689f9c8a54fd55bd38606aa6a8595ad213d4c9c9f9aca3fb217069a41028


好了，有了这个seed之后怎么办？这个还需要 BIP0032 来帮助了。

## BIP0032 分层确定性钱包

如上，我们有了一个seed，原则上，我们把这个seed直接处理一下，就可以得到一个私钥，但是 BIP0032 给出了一个更好的方案，如何由一个seed确定性的得到很多的私钥。BIP0032其实解决了一个问题，那就是你可以只备份一次，就可以生成无限多个地址来使用。要知道，在早期，因为忘了及时备份，丢失新生成的地址上的币是很频繁的。历史上，比特币官方客户端在打币的时候，每次都会为找零生成新地址，这就更加剧了丢币的行为。

首先我们根据seed生成一个主私钥以及主公钥

In [14]:
# 为了方便，我们直接使用一个固定的seed
seed = 0xfffcf9f6f3f0edeae7e4e1dedbd8d5d2cfccc9c6c3c0bdbab7b4b1aeaba8a5a29f9c999693908d8a8784817e7b7875726f6c696663605d5a5754514e4b484542

I = my_hmac(b"Bitcoin seed", seed.to_bytes(64, 'big'))
I_L = I[:32]
I_R = I[32:]
master_private_key = int.from_bytes(I_L, 'big')
master_chain_code = int.from_bytes(I_R, 'big')
master_public_point = priv_key_to_pub_key(master_private_key)
master_public_key = (b'\x02' if (master_public_point[1] & 1) == 0 else b'\x03') + master_public_point[0].to_bytes(32, 'big')
tmp = hashlib.new('ripemd160')
tmp.update(hashlib.sha256(master_public_key).digest())
hash160_master_public_key = tmp.digest()
master_fingerprint = int.from_bytes(hash160_master_public_key[:4], 'big')

def serialize_mainnet_private_key(private_key, chain_code, depth=0, parent_fingerprint=0, child_number=0):
    ret = (0x0488ADE4).to_bytes(4, 'big') # version
    ret += depth.to_bytes(1, 'big') # depth
    ret += parent_fingerprint.to_bytes(4, 'big') # parent fingerprint
    ret += child_number.to_bytes(4, 'big') # child number
    ret += chain_code.to_bytes(32, 'big') # chain code
    ret += b'\x00' + private_key.to_bytes(32, 'big') # private key
    checksum = hashlib.sha256(ret).digest()
    checksum = hashlib.sha256(checksum).digest()
    ret += checksum[:4] # checksum
    return ret

def serialize_mainnet_public_point(public_point, chain_code, depth=0, parent_fingerprint=0, child_number=0):
    ret = (0x0488B21E).to_bytes(4, 'big') # version
    ret += depth.to_bytes(1, 'big') # depth
    ret += parent_fingerprint.to_bytes(4, 'big') # parent fingerprint
    ret += child_number.to_bytes(4, 'big') # child number
    ret += chain_code.to_bytes(32, 'big') # chain code
    if public_point[1] & 1:
        ret += b'\x03'
    else:
        ret += b'\x02'
    ret += public_point[0].to_bytes(32, 'big') # public key
    checksum = hashlib.sha256(ret).digest()
    checksum = hashlib.sha256(checksum).digest()
    ret += checksum[:4] # checksum
    return ret

serialized_mainnet_private_key = base58.b58encode(serialize_mainnet_private_key(master_private_key, master_chain_code)).decode('utf-8')
print(f"master_private_key: {serialized_mainnet_private_key}")
assert serialized_mainnet_private_key == "xprv9s21ZrQH143K31xYSDQpPDxsXRTUcvj2iNHm5NUtrGiGG5e2DtALGdso3pGz6ssrdK4PFmM8NSpSBHNqPqm55Qn3LqFtT2emdEXVYsCzC2U"

serialized_mainnet_public_point = base58.b58encode(serialize_mainnet_public_point(master_public_point, master_chain_code)).decode('utf-8')
print(f"master_public_point: {serialized_mainnet_public_point}")
assert serialized_mainnet_public_point == "xpub661MyMwAqRbcFW31YEwpkMuc5THy2PSt5bDMsktWQcFF8syAmRUapSCGu8ED9W6oDMSgv6Zz8idoc4a6mr8BDzTJY47LJhkJ8UB7WEGuduB"


master_private_key: xprv9s21ZrQH143K31xYSDQpPDxsXRTUcvj2iNHm5NUtrGiGG5e2DtALGdso3pGz6ssrdK4PFmM8NSpSBHNqPqm55Qn3LqFtT2emdEXVYsCzC2U
master_public_point: xpub661MyMwAqRbcFW31YEwpkMuc5THy2PSt5bDMsktWQcFF8syAmRUapSCGu8ED9W6oDMSgv6Zz8idoc4a6mr8BDzTJY47LJhkJ8UB7WEGuduB


上面生成的主私钥和公钥我们成为 **m** (master的意思)。那我们怎么派生出更多的私钥和公钥呢？

下面我们来生成 **m/0** (这个类似一个路径的感觉)。

In [15]:
child_number = 0
I = my_hmac(master_chain_code.to_bytes(32, 'big'), master_public_key + child_number.to_bytes(4, 'big'))
# 核心在这里，父节点的私钥影响了字节点的私钥
child_private_key = (int.from_bytes(I[:32], 'big') + master_private_key) % secp256k1_n
child_chain_code = int.from_bytes(I[32:], 'big')
child_public_point = priv_key_to_pub_key(child_private_key)
child_public_key = (b'\x02' if (child_public_point[1] & 1) == 0 else b'\x03') + child_public_point[0].to_bytes(32, 'big')
tmp = hashlib.new('ripemd160')
tmp.update(hashlib.sha256(child_public_key).digest())
hash160_child_public_key = tmp.digest()
child_fingerprint = int.from_bytes(hash160_child_public_key[:4], 'big')

serialized_mainnet_child_private_key = base58.b58encode(serialize_mainnet_private_key(child_private_key, child_chain_code, 1, master_fingerprint, child_number)).decode('utf-8')
print(f"child_private_key: {serialized_mainnet_child_private_key}")
assert serialized_mainnet_child_private_key == "xprv9vHkqa6EV4sPZHYqZznhT2NPtPCjKuDKGY38FBWLvgaDx45zo9WQRUT3dKYnjwih2yJD9mkrocEZXo1ex8G81dwSM1fwqWpWkeS3v86pgKt"

serialized_mainnet_child_public_point = base58.b58encode(serialize_mainnet_public_point(child_public_point, child_chain_code, 1, master_fingerprint, child_number)).decode('utf-8')
print(f"child_public_point: {serialized_mainnet_child_public_point}")
assert serialized_mainnet_child_public_point == "xpub69H7F5d8KSRgmmdJg2KhpAK8SR3DjMwAdkxj3ZuxV27CprR9LgpeyGmXUbC6wb7ERfvrnKZjXoUmmDznezpbZb7ap6r1D3tgFxHmwMkQTPH"


child_private_key: xprv9vHkqa6EV4sPZHYqZznhT2NPtPCjKuDKGY38FBWLvgaDx45zo9WQRUT3dKYnjwih2yJD9mkrocEZXo1ex8G81dwSM1fwqWpWkeS3v86pgKt
child_public_point: xpub69H7F5d8KSRgmmdJg2KhpAK8SR3DjMwAdkxj3ZuxV27CprR9LgpeyGmXUbC6wb7ERfvrnKZjXoUmmDznezpbZb7ap6r1D3tgFxHmwMkQTPH


如上，我们就可以得到很多的衍生私钥和公钥，他们可以识别为

- **m**
- **m/0** , **m/1** , ...
- **m/0/0** , ... , **m/1/0**
- ... 

在上述的衍生过程中，我们已知父节点的私钥以及公钥，从而可以容易的衍生出子节点的私钥以及公钥。这里更有意思的是，如果我们只知道父节点的公钥，我们能够得到子节点的信息么？为什么这个很重要，可以考虑这样的应用场景。父节点（或者主节点）的私钥存在冷钱包里面，但是其公钥是放在热钱包里面的，方便收款和查看余额，如果能够直接从父节点的公钥得到子节点的公钥，那我们就可以方便创建更多地址了。

如果理解一下上面的衍生过程，我们可以看出，这个想法是可以实现的。上面的衍生过程可以看作这样几个子过程:

1. master_public_point, master_chain_code -> I
2. I -> child_chain_code
3. I + master_private_key -> child_private_key
4. child_private_key -> child_public_point

3和4可以结合起来得到:

1. master_public_point, master_chain_code -> I
2. I -> child_chain_code
3. I + master_public_point -> child_public_point

In [16]:
new_I = my_hmac(master_chain_code.to_bytes(32, 'big'), master_public_key + child_number.to_bytes(4, 'big'))
new_child_public_point = secp256k1_add(priv_key_to_pub_key(int.from_bytes(new_I[:32], 'big')), master_public_point)
assert new_child_public_point == child_public_point

实际上，BIP0032还搞了一种衍生机制，我猜可能是为隐私的考虑。因为如上所述，从父节点的公钥就可以推测出所有子节点的公钥，那我知道了你的公钥，就可以容易的推算出所有子节点、孙子节点的公钥，从而查看你的余额，一点隐私都没有了。

这种新的机制把 **child_number >= 2^31** 单独拿出来，加以隐私考虑，为了书写方便，这个衍生方式用 **m/0'** 来代表 **child_number=2^31** 。

下面是 **m/0/2147483647'**

In [17]:
grandson_number = 2147483647 + 2**31
# 注意这里的区别，如果不知道父节点的私钥，我们是得不到I的数据的
I = my_hmac(child_chain_code.to_bytes(32, 'big'), b'\x00' + child_private_key.to_bytes(32, 'big') + grandson_number.to_bytes(4, 'big'))
grandson_private_key = (int.from_bytes(I[:32], 'big') + child_private_key) % secp256k1_n
grandson_chain_code = int.from_bytes(I[32:], 'big')
grandson_public_point = priv_key_to_pub_key(grandson_private_key)
grandson_public_key = (b'\x02' if (grandson_public_point[1] & 1) == 0 else b'\x03') + grandson_public_point[0].to_bytes(32, 'big')
tmp = hashlib.new('ripemd160')
tmp.update(hashlib.sha256(grandson_public_key).digest())
hash160_grandson_public_key = tmp.digest()
grandson_fingerprint = int.from_bytes(hash160_grandson_public_key[:4], 'big')

serialized_mainnet_grandson_private_key = base58.b58encode(serialize_mainnet_private_key(grandson_private_key, grandson_chain_code, 2, child_fingerprint, grandson_number)).decode('utf-8')
print(f"grandson_private_key: {serialized_mainnet_grandson_private_key}")
assert serialized_mainnet_grandson_private_key == "xprv9wSp6B7kry3Vj9m1zSnLvN3xH8RdsPP1Mh7fAaR7aRLcQMKTR2vidYEeEg2mUCTAwCd6vnxVrcjfy2kRgVsFawNzmjuHc2YmYRmagcEPdU9"

serialized_mainnet_grandson_public_point = base58.b58encode(serialize_mainnet_public_point(grandson_public_point, grandson_chain_code, 2, child_fingerprint, grandson_number)).decode('utf-8')
print(f"grandson_public_point: {serialized_mainnet_grandson_public_point}")
assert serialized_mainnet_grandson_public_point == "xpub6ASAVgeehLbnwdqV6UKMHVzgqAG8Gr6riv3Fxxpj8ksbH9ebxaEyBLZ85ySDhKiLDBrQSARLq1uNRts8RuJiHjaDMBU4Zn9h8LZNnBC5y4a"


grandson_private_key: xprv9wSp6B7kry3Vj9m1zSnLvN3xH8RdsPP1Mh7fAaR7aRLcQMKTR2vidYEeEg2mUCTAwCd6vnxVrcjfy2kRgVsFawNzmjuHc2YmYRmagcEPdU9
grandson_public_point: xpub6ASAVgeehLbnwdqV6UKMHVzgqAG8Gr6riv3Fxxpj8ksbH9ebxaEyBLZ85ySDhKiLDBrQSARLq1uNRts8RuJiHjaDMBU4Zn9h8LZNnBC5y4a


## bither.net钱包

<https://bither.net>

基于版本1.4.8: 4d8770d4c24620bf40559ae402148d526ea488b0069b7347fe0903fec9a73b51 bither-desktop.jar

做个小实验。首先我们用bither生成一个HDM热钱包，相关数据如下。根据 BIP0044，我们猜测这个地址的衍生路径是 **m/44'/0'/0'/0/0** ，我们来验证一下。


In [18]:
bither_mnemnic_words = "peanut-test-swear-eager-obtain-tuition-plate-tool-butter-dragon-identify-army".replace('-', ' ')
bither_mnemnic_words_as_idx_list = [mnemonic_word_list.index(word) for word in bither_mnemnic_words.split(' ')]
bither_mnemnic_words_as_bin_string = ''.join([f"{idx:011b}" for idx in bither_mnemnic_words_as_idx_list])
bither_mnemnic_words_as_int = int(bither_mnemnic_words_as_bin_string, 2)
bither_mnemnic_words_as_int >>= 4
print(f"bither_mnemnic_words_as_int: 0x{bither_mnemnic_words_as_int:032x}")
bither_first_btc_address = "1Ntu2p29uSgkjvHihLrDvnKwWxx7VnZ6Lv"

def get_fingerprint(compressed_public_key):
    tmp = hashlib.new('ripemd160')
    tmp.update(hashlib.sha256(compressed_public_key).digest())
    return int.from_bytes(tmp.digest()[:4], 'big')

def get_address(compressed_public_key):
    hashed_pub_key = hashlib.sha256(compressed_public_key).digest()

    ripemd160_hasher = hashlib.new('ripemd160')
    ripemd160_hasher.update(hashed_pub_key)
    ripemd160_pub_key = ripemd160_hasher.digest()

    ripemd160_pub_key_with_version = int.from_bytes(ripemd160_pub_key, 'big').to_bytes(21, 'big') # 添加版本号

    hashed_ripemd160_pub_key_with_version = hashlib.sha256(ripemd160_pub_key_with_version).digest()

    hashed_twice_ripemd160_pub_key_with_version = hashlib.sha256(hashed_ripemd160_pub_key_with_version).digest()

    checksum = hashed_twice_ripemd160_pub_key_with_version[:4]

    address = ripemd160_pub_key_with_version + checksum
    return base58.b58encode(address).decode('utf-8')

def mnemonic_to_key(mnemonic, path, extra_phrase=""):
    # extra_phrase used later
    seed = my_pbkdf2(mnemonic.encode('utf-8'), ("mnemonic" + extra_phrase).encode('utf-8'))
    I = my_hmac(b"Bitcoin seed", seed)
    I_L = I[:32]
    I_R = I[32:]
    private_key = int.from_bytes(I_L, 'big')
    chain_code = int.from_bytes(I_R, 'big')
    public_key = priv_key_to_pub_key(private_key)
    compressed_public_key = (b'\x02' if (public_key[1] & 1) == 0 else b'\x03') + public_key[0].to_bytes(32, 'big')
    fingerprint = get_fingerprint(compressed_public_key)

    for child_number in path:
        if child_number < 2**31:
            # normal child
            I = my_hmac(chain_code.to_bytes(32, 'big'), compressed_public_key + child_number.to_bytes(4, 'big'))
        else:
            # hardened child
            I = my_hmac(chain_code.to_bytes(32, 'big'), b'\x00' + private_key.to_bytes(32, 'big') + child_number.to_bytes(4, 'big'))
        private_key = (int.from_bytes(I[:32], 'big') + private_key) % secp256k1_n
        chain_code = int.from_bytes(I[32:], 'big')
        public_key = priv_key_to_pub_key(private_key)
        compressed_public_key = (b'\x02' if (public_key[1] & 1) == 0 else b'\x03') + public_key[0].to_bytes(32, 'big')
        fingerprint = get_fingerprint(compressed_public_key)

    address = get_address(compressed_public_key)
    return {"private_key": private_key, "address": address, "seed": seed}

computed_bither_info = mnemonic_to_key(bither_mnemnic_words, [44+2**31, 0+2**31, 0+2**31, 0, 0])
computed_bither_first_btc_address = computed_bither_info["address"]
print(f"computed_bither_first_btc_address: {computed_bither_first_btc_address}")
assert computed_bither_first_btc_address == bither_first_btc_address

bither_mnemnic_words_as_int: 0xa1fbf36e227989d4698f251f4841c206
computed_bither_first_btc_address: 1Ntu2p29uSgkjvHihLrDvnKwWxx7VnZ6Lv


地址果然是对的。

bither钱包除了可以导出助记词之外，实际上还可以导出一个 **HD账户种子二维码** 的东西，这玩意儿是加密过的种子，但是要注意是生成助记词的那个随机数，用这个随机数就可以生成助记词。

In [19]:
bither_seed_qrcode_content = "3D25EF489FCEA9BD9470C8254E65DD3AC4C6AFCD60882F3050065B2CB16268FF/DF48C3E83CB3D59707F87B6544AE6BF6/03FB2B4121FBDB4E07"
bither_encrypted_seed = bytes.fromhex(bither_seed_qrcode_content.split('/')[0])
bither_iv = bytes.fromhex(bither_seed_qrcode_content.split('/')[1])
bither_salt = bytes.fromhex(bither_seed_qrcode_content.split('/')[2])
bither_password = b"\x001\x002\x003\x004\x005\x00q\x00w\x00e\x00r\x00t" # 我们随便设置一个密码，注意这里的编码 WTF

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

def encrypt_bither_seed(seed, password, iv, salt):
    salt = salt[1:]
    assert len(seed) == 16
    assert len(iv) == 16
    assert len(salt) == 8

    aes_key = hashlib.scrypt(password, salt=salt, n=16384, r=8, p=1, dklen=32)

    cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv), backend=default_backend())

    encryptor = cipher.encryptor()

    padder = padding.PKCS7(algorithms.AES.block_size).padder()

    padded_data = padder.update(seed) + padder.finalize()
    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()

    return encrypted_data

ret = encrypt_bither_seed(bither_mnemnic_words_as_int.to_bytes(16, "big"), bither_password, bither_iv, bither_salt)
assert ret == bither_encrypted_seed

def decrypt_bither_seed(encrypted_seed, password, iv, salt):
    assert salt[0] == 1 + 2 # IS_COMPRESSED_FLAG + IS_FROMXRANDOM_FLAG
    salt = salt[1:]
    assert len(encrypted_seed) == 32
    assert len(iv) == 16
    assert len(salt) == 8

    aes_key = hashlib.scrypt(password, salt=salt, n=16384, r=8, p=1, dklen=32)

    cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv), backend=default_backend())

    decryptor = cipher.decryptor()

    decrypted_padded_bytes = decryptor.update(encrypted_seed) + decryptor.finalize()
    assert len(decrypted_padded_bytes) == 32

    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()

    decrypted_bytes = unpadder.update(decrypted_padded_bytes) + unpadder.finalize()

    return decrypted_bytes

ret = decrypt_bither_seed(bither_encrypted_seed, bither_password, bither_iv, bither_salt)
#print(f"decrypted_mnemonic_seeds: 0x{int.from_bytes(ret, 'big'):032x}")
assert ret == bither_mnemnic_words_as_int.to_bytes(16, "big")

### bither非HD模式

除了上述的这种复杂的所谓HD模式（先生成助记词，然后生成一个seed，然后再生成各个地址）之外，bither还有一种简单直接的模式，就是独立的一个一个的地址，相互没有任何关系。

在这种模式之下，用户的私钥可以直接导出，也可以先加密后导出。

<https://en.bitcoin.it/wiki/Wallet_import_format>

In [20]:
bither_normal_address = "1MnqqRyVjvRJKJE1GjWexbK2E8jatwyVYW"
# WIF private key for compressed public key
bither_normal_address_private_key_str = "L5AjnXZbfZXTwqyYZbEtpfKiUtWbfxHC14djNixeoML9bigbt6WV"

def wif_to_address(wif):
    # first byte is 0x80 for mainnet
    # last 4 bytes are the checksum
    # last 5th byte is 0x01 for compressed public key
    private_key = int.from_bytes(base58.b58decode(wif)[1:-5], 'big')
    public_point = priv_key_to_pub_key(private_key)
    compressed_public_key = (b'\x02' if (public_point[1] & 1) == 0 else b'\x03') + public_point[0].to_bytes(32, 'big')
    return get_address(compressed_public_key)

assert wif_to_address(bither_normal_address_private_key_str) == bither_normal_address

bither_encrypted_wif = "E8BE291DD03FBDBBFC916348EB37160EEFF43B1ADF7122C4557E4BA2792078726FCE930FA9755A417C95B8127A16E332/9C118D1BA7E529E66E4993D064E33FBE/0376CBFC3EB2B0E8BA"
bither_encrypted_wif = bither_encrypted_wif.split('/')
bither_encrypted_wif_bytes = bytes.fromhex(bither_encrypted_wif[0])
bither_encrypted_wif_iv = bytes.fromhex(bither_encrypted_wif[1])
bither_encrypted_wif_salt = bytes.fromhex(bither_encrypted_wif[2])

def decrypt_bither_wif(encrypted_wif, password, iv, salt):
    assert salt[0] == 1 + 2 # IS_COMPRESSED_FLAG + IS_FROMXRANDOM_FLAG
    salt = salt[1:]
    assert len(encrypted_wif) == 48
    assert len(iv) == 16
    assert len(salt) == 8

    aes_key = hashlib.scrypt(password, salt=salt, n=16384, r=8, p=1, dklen=32)

    cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv), backend=default_backend())

    decryptor = cipher.decryptor()

    decrypted_padded_bytes = decryptor.update(encrypted_wif) + decryptor.finalize()
    assert len(decrypted_padded_bytes) == 48

    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()

    decrypted_bytes = unpadder.update(decrypted_padded_bytes) + unpadder.finalize()
    assert len(decrypted_bytes) == 32

    return decrypted_bytes

wif_bytes = decrypt_bither_wif(bither_encrypted_wif_bytes, bither_password, bither_encrypted_wif_iv, bither_encrypted_wif_salt)
expected_wif_bytes = base58.b58decode(bither_normal_address_private_key_str)[1:-5]
assert wif_bytes == expected_wif_bytes

## Ledger Nano S Plus

有了bither.net的经验，貌似一切就容易了。注意参考 <https://en.bitcoin.it/wiki/BIP_0044> 。我们来初始化一个Ledger设备，然后检查一切是不是如同 bither.net 一样。

In [21]:
ledger_mnemonic_words = "fabric gaze exchange canvas symbol use swamp arctic resemble label guide angry nature solve laptop exit fuel result hood cactus piece hotel divorce illegal"
ledger_first_btc_address_info_from_computed = mnemonic_to_key(ledger_mnemonic_words, [44+2**31, 0+2**31, 0+2**31, 0, 0])
ledger_first_btc_address_shown = "16MoycEPk6ZUGnrR5EkmNJgkap1hnLKcVr"
assert ledger_first_btc_address_info_from_computed["address"] == ledger_first_btc_address_shown

看得出来，ledger和bither之间毫无区别，都是遵守同样几个BIP，以及相同的衍生path，整个计算过程也毫无二致。

不过ledger还有个一个高级功能，就是可以设置一个密码，从而得到完全不同的地址集合。我们来试一下

In [22]:
ledger_extra_password = "12345"
ledger_extra_btc_address_info_from_computed = mnemonic_to_key(ledger_mnemonic_words, [44+2**31, 0+2**31, 0+2**31, 0, 0], extra_phrase=ledger_extra_password)
ledger_extra_btc_address_shown = "1PB24uzHD786A5x6xSKjLtBBewnvapC2rk"
assert ledger_extra_btc_address_info_from_computed["address"] == ledger_extra_btc_address_shown

## 手搓BTC

由上可知，如果遵循BIP0039，我们实际上需要做的是，首先生成若干比特的随机数，然后从2048个助记词中挑选出助记词列表。

如果我们不相信钱包的随机数生成机制，我们是不是可以手工完成上述过程？貌似是可以的，但需要仔细分析一下。

对Ledger钱包而言，我们要生成的助记词是24个，对应 ```24 * 11 = 264``` 比特，其中 256 比特来自随机数，另外8比特是校验码，也就是checksum。这个校验码是通过sha256哈希计算而得的，这个部分我们要手工计算就困难了。

类似分析bither钱包，我们要生成的助记词是12个，对应 ```12 * 11 = 132``` 比特，其中 128 比特来自随机数，另外4比特是校验码。这个校验码同样是通过哈希计算得到的，应该也很难于进行手工计算，对吧？但是，相对于计算哈希，我们可以直接穷举所有的 ```2 ^ 4 = 16``` 种不同的校验码，似乎也不算太多？

我们来试一下：

In [23]:
# 我们通过摇骰子来得到随机性，每次摇骰子，如果得到的数字是 5和6 就丢弃，重新摇骰子
dice_results = [3,2,3,3,1,4,2,2,2,2,1,4,1,2,2,1,2,4,3,3,2,3,3,3,3,4,1,2,1,2,3,1,1,3,1,1,2,1,3,3,1,2,2,2,4,4,3,1,1,3,2,4,2,4,1,1,2,1,1,2,2,2,2,2]
dice_entropy = int(''.join([str(x-1) for x in dice_results]), 4)

for checksum in range(2**4):
    entropy = dice_entropy * 2**4 + checksum
    mnemonic = []
    for i in range(12):
        mnemonic.append(mnemonic_word_list[entropy & 2047])
        entropy = entropy >> 11
    mnemonic = mnemonic[::-1]
    mnemonic = ' '.join(mnemonic)
    first_address = mnemonic_to_key(mnemonic, [44+2**31, 0+2**31, 0+2**31, 0, 0])["address"]
    print(f"mnemonic for checksum={checksum:x}: {mnemonic} \tfirst_address: {first_address}")


mnemonic for checksum=0: olive pride shallow visit prison metal license dress wealth describe aware present 	first_address: 1HtP3Ck78jbbvDDFBuve1jjTJE7t64mDyC
mnemonic for checksum=1: olive pride shallow visit prison metal license dress wealth describe aware pretty 	first_address: 1MX49nSeQ1DXAa5g2Qech1DrshghM7QS8C
mnemonic for checksum=2: olive pride shallow visit prison metal license dress wealth describe aware prevent 	first_address: 1CsnKcS3sUrivoThpW5tvr97nszaMJVtxt
mnemonic for checksum=3: olive pride shallow visit prison metal license dress wealth describe aware price 	first_address: 1MVx4x8WzDPTemkSbYuGJS14EGVqL53Lig
mnemonic for checksum=4: olive pride shallow visit prison metal license dress wealth describe aware pride 	first_address: 1FpF2kY7nZk5jc7fZCob6XkGQMKmexb7V4
mnemonic for checksum=5: olive pride shallow visit prison metal license dress wealth describe aware primary 	first_address: 1A4RYyEHcgyp6vPH8fvbbcX7rz8VWokMXd
mnemonic for checksum=6: olive pride shallow visit 

通过手工将上面16组可能的助记词输入到bither钱包中去，我们将知道哪一个校验码才是正确的。

通过不断尝试，答案是几乎到了列表末尾的 ```checksum=e```，但是相比计算sha256哈希来说，这个也算是简单多了。

## multi-sig多签存储btc

如果只使用单独的一个钱包来存币，上面介绍的机制貌似已经够了。但有时候我们不免会有一个疑问，如果这个钱包出问题了怎么办？比如要是bither钱包出bug了，比如随机数生成过程出问题了，或者是签名交易的时候出bug泄漏私钥了，怎么办？要是Ledger钱包固件出问题了，比如官方被黑了，固件出bug了，怎么办？一个容易想到的答案是，如果我们能够同时使用这两个钱包，甚至多个钱包的安全性，似乎会更安全一些？

这个可以用比特币的多签机制来解决。下面直接用一个例子来解释。

<https://en.bitcoin.it/wiki/OP_CHECKMULTISIG>

<https://mempool.space/zh/tx/d3adb18d5e118bb856fbea4b1af936602454b44a98fc6c823aedc858b491fc13> 是一个 ```2/3``` 的例子。

In [24]:
# raw data from https://bitaps.com/raw/transaction/d3adb18d5e118bb856fbea4b1af936602454b44a98fc6c823aedc858b491fc13

ms_recv_tx = {
    "version": 0x01, # raw is 01000000
    "flag": None, # for legacy tx, flag is None
    "in_count": 1, # raw is 01
    "input": [
        {
            "tx": 0x1b16e4a8af0831da62d8baae47636e49060049948a3d9b8b9b78eafb853b5a2b,
            "index": 1, # raw is 01000000
            "script_size": 0x8b, # raw is 8b
            "script_sig": 0x48304502204c3da378d8323a7233892b8050f738da69daf765ddc0d9815d9ad352286b70c2022100dff11c19338daa85ec5b124434de3b4378ebe8c56950348d5f66197af1d04f09014104910ae6c9b41b04d366ea54e920663c691843bb83ef7336cd6a0f79b0ac82ee38d2ca23a24adc2348d82c8ca13f0db885712493e89d88551118a7e80ff66ab23c,
            "seq_no": 0xffffffff, # raw is ffffffff
        }
    ],
    "out_count": 1, # raw is 01
    "output": [
        {
            "value": 9980000, # raw is 6048980000000000
            "script_size": 0x17, # raw is 17
            "script": 0xa914b4acb9d78d6a6256964a60484c95de490eaaae7587,
        }
    ],
    "lock_time": 0x00, # raw is 00000000
}

# 主要看 output[0].script
# a914b4acb9d78d6a6256964a60484c95de490eaaae7587
output_script = [
    "OP_HASH160", # 0xa9
    "0x14", # 20 bytes
    "0xb4acb9d78d6a6256964a60484c95de490eaaae75", # 20 bytes
    "OP_EQUAL", # 0x87
]

关键是看这个 ```0xb4acb9d78d6a6256964a60484c95de490eaaae75``` 代表的是什么。

首先参考 BIP0013 和 BIP0016，我们可以算出这个hash对应的收款地址是：

In [25]:
p2sh_script_hash = 0xb4acb9d78d6a6256964a60484c95de490eaaae75
p2sh_address = "3JALUHKvqB7NToPA2jALntCUWmvsgYMyGj"

def p2sh_script_hash_to_address(p2sh_script_hash):
    hash_as_bytes = p2sh_script_hash.to_bytes(20, 'big')
    bytes_with_version = b'\x05' + hash_as_bytes
    checksum = hashlib.sha256(hashlib.sha256(bytes_with_version).digest()).digest()[:4]
    return base58.b58encode(bytes_with_version + checksum).decode('utf-8')

assert p2sh_script_hash_to_address(p2sh_script_hash) == p2sh_address

那这个脚本具体代表什么呢，光从这个收币的交易是看不出来的，还得从花费这些币的交易中所泄漏的信息才能看出来。

<https://mempool.space/zh/tx/cc11ca9e9dc188663c41eb23b15370f68eded56b7ec54dd5bc4f2d2ae93addb2>

In [26]:
# raw data at https://bitaps.com/raw/transaction/cc11ca9e9dc188663c41eb23b15370f68eded56b7ec54dd5bc4f2d2ae93addb2

ms_send_tx = {
    "version": 0x01, # raw is 01000000
    "flag": None, # for legacy tx, flag is None
    "in_count": 1, # raw is 01
    "inputs": [
        {
            "tx": 0x13fc91b458c8ed3a826cfc984ab454246036f91a4beafb56b88b115e8db1add3,
            "index": 0, # raw is 00000000
            "script_size": 0x015f, # raw is fd5f01
            "scrpt": 0x00483044022018b88c05ab571cf31bf317a98ed5909cf43218f472bfbeb82b6857d3b1edf4ee0220686627e66b368e298114a097b5814a76fdaac23f7728a42d38415552ba68c8220101493046022100b7a70f4c3b2b5d24475f9664bb77b6046c0251c89b446aad8a86b584b74ed414022100d53dd27b741801a908fdf7c2ee259f444f549b56d7a6c682599ef67b1edb6d24014cc9524104f3d35132084eb1b99b6506178c20adb42d26296012e452e392689bdb6553db33ba24b900000892805de1646821c7b0fb50b3d879c26e2b493b7041e6215356a04104ab4ecc9e8ea2da0562af25bcaede00c4d5a00db60edc17672376decf0a35a34fdc9f1ffad1fb74fd7b1b198b9231c25df88e0769bec49975649b4b3f40adafb04104f7149f270717c00f6cc09b9ce3c22791c4aab1af40a5107aacca85b6f644cc0d84459e308f998d801b8d9d355f8ec33b0e41866841e2870754cf667a9821703d53ae,
            "seq_no": 0xffffffff, # raw is ffffffff
        }
    ],
    "out_count": 1, # raw is 01
    "outputs": [
        {
            "value": 9960000, # raw is 40fa970000000000
            "script_size": 0x17, # raw is 17
            "script": 0xa9146ebdbaf0840274a6b23b0643b75ef4e8e24f37b887,
        }
    ],
}

input_script = [
    "OP_0",
    "0x48", # 72 bytes
    0x3044022018b88c05ab571cf31bf317a98ed5909cf43218f472bfbeb82b6857d3b1edf4ee0220686627e66b368e298114a097b5814a76fdaac23f7728a42d38415552ba68c8220101,
    "0x49", # 73 bytes
    0x3046022100b7a70f4c3b2b5d24475f9664bb77b6046c0251c89b446aad8a86b584b74ed414022100d53dd27b741801a908fdf7c2ee259f444f549b56d7a6c682599ef67b1edb6d2401,
    "OP_PUSHDATA1", # 0x4c
    "0xc9", # 0xc9 bytes as the length
    0x524104f3d35132084eb1b99b6506178c20adb42d26296012e452e392689bdb6553db33ba24b900000892805de1646821c7b0fb50b3d879c26e2b493b7041e6215356a04104ab4ecc9e8ea2da0562af25bcaede00c4d5a00db60edc17672376decf0a35a34fdc9f1ffad1fb74fd7b1b198b9231c25df88e0769bec49975649b4b3f40adafb04104f7149f270717c00f6cc09b9ce3c22791c4aab1af40a5107aacca85b6f644cc0d84459e308f998d801b8d9d355f8ec33b0e41866841e2870754cf667a9821703d53ae
]

# 主要看 input_script[7]，也就是 0x5241...
p2sh_redeem_script = [
    "OP_2", # 0x52
    "0x41", # 65 bytes
    0x04f3d35132084eb1b99b6506178c20adb42d26296012e452e392689bdb6553db33ba24b900000892805de1646821c7b0fb50b3d879c26e2b493b7041e6215356a0, # 65 bytes
    "0x41", # 65 bytes
    0x04ab4ecc9e8ea2da0562af25bcaede00c4d5a00db60edc17672376decf0a35a34fdc9f1ffad1fb74fd7b1b198b9231c25df88e0769bec49975649b4b3f40adafb0, # 65 bytes
    "0x41", # 65 bytes
    0x04f7149f270717c00f6cc09b9ce3c22791c4aab1af40a5107aacca85b6f644cc0d84459e308f998d801b8d9d355f8ec33b0e41866841e2870754cf667a9821703d, # 65 bytes
    "OP_3", # 0x53
    "OP_CHECKMULTISIG", # 0xae
]
p2sh_redeem_script_as_bytes = input_script[7].to_bytes(0xc9, "big")
tmp = hashlib.sha256(p2sh_redeem_script_as_bytes).digest()
tmp_hasher = hashlib.new('ripemd160')
tmp_hasher.update(tmp)
computed_p2sh_script_hash = tmp_hasher.digest()
assert computed_p2sh_script_hash == p2sh_script_hash.to_bytes(20, 'big')

从上面可以看出，这个 ```2/3``` 多签地址的三个公钥以及对应地址分别是

In [27]:
pub_key_1 = 0x04f3d35132084eb1b99b6506178c20adb42d26296012e452e392689bdb6553db33ba24b900000892805de1646821c7b0fb50b3d879c26e2b493b7041e6215356a0
pub_key_2 = 0x04ab4ecc9e8ea2da0562af25bcaede00c4d5a00db60edc17672376decf0a35a34fdc9f1ffad1fb74fd7b1b198b9231c25df88e0769bec49975649b4b3f40adafb0
pub_key_3 = 0x04f7149f270717c00f6cc09b9ce3c22791c4aab1af40a5107aacca85b6f644cc0d84459e308f998d801b8d9d355f8ec33b0e41866841e2870754cf667a9821703d

compressed_pub_key_1 = (b'\x02' if (pub_key_1 & 1) == 0 else b'\x03') + ((pub_key_1>>256) % 2**256).to_bytes(32, 'big')
compressed_pub_key_2 = (b'\x02' if (pub_key_2 & 1) == 0 else b'\x03') + ((pub_key_2>>256) % 2**256).to_bytes(32, 'big')
compressed_pub_key_3 = (b'\x02' if (pub_key_3 & 1) == 0 else b'\x03') + ((pub_key_3>>256) % 2**256).to_bytes(32, 'big')

address_1 = get_address(compressed_pub_key_1)
address_2 = get_address(compressed_pub_key_2)
address_3 = get_address(compressed_pub_key_3)

print(f"address_1: {address_1}")
print(f"address_2: {address_2}")
print(f"address_3: {address_3}")


address_1: 1BhbbJKpn83Npz1qL1ngexnXASyNBn5Wsc
address_2: 1EtfNeupjsjq5Dn2eNEEQp2DaFgfPFSBdp
address_3: 1HbPBdgvD8GbYedhZxAtBMxE8TkWVQFfii


为了把收到的币发走，上面的交易使用了3个私钥中的2个，这个体现在两个签名上。

In [28]:
# 先看看bitcoin message签名的例子

def secp256k1_mul(v, point):
    assert v < 2**256
    ret = None
    base = point
    for i in range(256):
        if v & (1 << i):
            ret = secp256k1_add(ret, base)
        base = secp256k1_add(base, base)
    return ret    

# 测试bitcoin wallet对消息的签名
def secp256k1_check_msg_signature(pub_key, sig, msg, header):
    r = sig[0]
    s = sig[1]
    assert 1 <= r < secp256k1_n and 1 <= s < secp256k1_n

    assert len(msg) < 250
    to_verify_str = b"\x18Bitcoin Signed Message:\n" + len(msg).to_bytes(1, 'big') + msg
    to_verify_str_hash = hashlib.sha256(to_verify_str).digest()
    to_verify_str_hash = hashlib.sha256(to_verify_str_hash).digest()

    z = int.from_bytes(to_verify_str_hash, 'big')

    x = r
    x2 = (x ** 3 + 7) % secp256k1_p
    assert secp256k1_p % 4 == 3
    y = pow(x2, (secp256k1_p+1)//4, secp256k1_p)
    assert y * y % secp256k1_p == x2

    # Compute the header byte. 
    # https://en.bitcoin.it/wiki/Message_signing @FIXME
    # If r < p-n and is_even(y), set HeaderByte to 30. @FIXME is_odd
    # If r < p-n and is_odd(y), set HeaderByte to 27.  @FIXME is_even
    # If r ≥ p-n and is_even(y), set HeaderByte to 28.  @FIXME is_odd?
    # If r ≥ p-n and is_odd(y), set HeaderByte to 29. @FIXME is_even
    if header == 0x1c: # 28
        assert r >= secp256k1_p - secp256k1_n
        if (y & 1) == 0:
            y = secp256k1_p - y
    else:
        assert False

    R = (x, y)
    e = (secp256k1_n*2 - z) % secp256k1_n

    r_inv = pow(r, -1, secp256k1_n)
    assert r * r_inv % secp256k1_n == 1

    t1 = s * r_inv % secp256k1_n
    t2 = e * r_inv % secp256k1_n
    PublicKey = secp256k1_add(secp256k1_mul(t1, R), secp256k1_mul(t2, (secp256k1_Gx, secp256k1_Gy)))
    assert PublicKey == pub_key


# test example (uncompressed public key)
to_test_priv_key = base58.b58decode("5KYZdUEo39z3FPrtuX2QbbwGnNP5zTd7yyr2SC1j299sBCnWjss")
to_test_priv_key = int.from_bytes(to_test_priv_key[1:-4])

to_test_pub_point = priv_key_to_pub_key(to_test_priv_key)
to_test_pub_key = b'\x04' + to_test_pub_point[0].to_bytes(32, 'big') + to_test_pub_point[1].to_bytes(32, 'big')
to_test_address = get_address(to_test_pub_key)
assert to_test_address == "1HZwkjkeaoZfTSaJxDw6aKkxp45agDiEzN"

to_test_msg = b"abc"

import base64
to_test_sig = base64.b64decode("HArNwRD6S0Z5xej3kMdPBaIe1p/w3avfFYD6EwZVhc5gUyDp8vu9MhvWP84s2vYQvuXVBRi2ev0Rp9sSYIcVgPA=")
assert len(to_test_sig) == 65
to_test_sig_header_byte = to_test_sig[0]
assert to_test_sig_header_byte == 0x1c # ECDSA verification, P2PKH uncompressed address
to_test_sig = to_test_sig[1:]
to_test_sig = int.from_bytes(to_test_sig, 'big')
to_test_r = to_test_sig >> 256
to_test_s = to_test_sig % 2**256

secp256k1_check_msg_signature(to_test_pub_point, (to_test_r, to_test_s), to_test_msg, to_test_sig_header_byte)


In [61]:
# 再来验证一下交易的签名

# not bip-0062??? @FIXME
sig_1 = 0x3044022018b88c05ab571cf31bf317a98ed5909cf43218f472bfbeb82b6857d3b1edf4ee0220686627e66b368e298114a097b5814a76fdaac23f7728a42d38415552ba68c82201 #@FIXME 01
sig_1_parsed = 0x18b88c05ab571cf31bf317a98ed5909cf43218f472bfbeb82b6857d3b1edf4ee686627e66b368e298114a097b5814a76fdaac23f7728a42d38415552ba68c822
# yes bip-0062
sig_2 = 0x3046022100b7a70f4c3b2b5d24475f9664bb77b6046c0251c89b446aad8a86b584b74ed414022100d53dd27b741801a908fdf7c2ee259f444f549b56d7a6c682599ef67b1edb6d2401
sig_2_parsed = 0xb7a70f4c3b2b5d24475f9664bb77b6046c0251c89b446aad8a86b584b74ed414d53dd27b741801a908fdf7c2ee259f444f549b56d7a6c682599ef67b1edb6d24

def get_sig_hash():
    raw = "01000000" #version 
    raw += "01" #in count 
    raw += "13fc91b458c8ed3a826cfc984ab454246036f91a4beafb56b88b115e8db1add3"  # in tx
    raw += "00000000"  # in index
    raw += "fd5f01"  # in script len
    raw += "00483044022018b88c05ab571cf31bf317a98ed5909cf43218f472bfbeb82b68"  # input_script
    raw += "57d3b1edf4ee0220686627e66b368e298114a097b5814a76fdaac23f7728a42d" 
    raw += "38415552ba68c8220101493046022100b7a70f4c3b2b5d24475f9664bb77b604" 
    raw += "6c0251c89b446aad8a86b584b74ed414022100d53dd27b741801a908fdf7c2ee" 
    raw += "259f444f549b56d7a6c682599ef67b1edb6d24014cc9"  # total  32 + 32 + 32 + 32 + 22 = 150 bytes till now
    raw += "524104f3d35132084eb1b99b6506178c20adb42d26296012e452e392689bdb65"  # starting of the redeem script
    raw += "53db33ba24b900000892805de1646821c7b0fb50b3d879c26e2b493b7041e621" 
    raw += "5356a04104ab4ecc9e8ea2da0562af25bcaede00c4d5a00db60edc17672376de" 
    raw += "cf0a35a34fdc9f1ffad1fb74fd7b1b198b9231c25df88e0769bec49975649b4b" 
    raw += "3f40adafb04104f7149f270717c00f6cc09b9ce3c22791c4aab1af40a5107aac" 
    raw += "ca85b6f644cc0d84459e308f998d801b8d9d355f8ec33b0e41866841e2870754cf667a9821703d53ae" 
    # other parts
    raw += "ffffffff0140fa97000000000017a9146ebdbaf0840274a6b23b0643b75ef4e8e24f37b88700000000"
    
    # the 150 bytes above should be removed and then hash it
    # remember to update the input script length
    modified_raw = "01000000" #version
    modified_raw += "01"  #in count
    modified_raw += "13fc91b458c8ed3a826cfc984ab454246036f91a4beafb56b88b115e8db1add3"  # in tx
    modified_raw += "00000000"  # in index
    modified_raw += "c9"  # in script len (updated here)
    modified_raw += "524104f3d35132084eb1b99b6506178c20adb42d26296012e452e392689bdb65"  # starting of the redeem script
    modified_raw += "53db33ba24b900000892805de1646821c7b0fb50b3d879c26e2b493b7041e621" 
    modified_raw += "5356a04104ab4ecc9e8ea2da0562af25bcaede00c4d5a00db60edc17672376de" 
    modified_raw += "cf0a35a34fdc9f1ffad1fb74fd7b1b198b9231c25df88e0769bec49975649b4b" 
    modified_raw += "3f40adafb04104f7149f270717c00f6cc09b9ce3c22791c4aab1af40a5107aac" 
    modified_raw += "ca85b6f644cc0d84459e308f998d801b8d9d355f8ec33b0e41866841e2870754cf667a9821703d53ae" 
    # other parts
    modified_raw += "ffffffff0140fa97000000000017a9146ebdbaf0840274a6b23b0643b75ef4e8e24f37b88700000000"

    hashType = 1
    modified_raw += hashType.to_bytes(4, 'little').hex()

    #print(modified_raw)

    hash = hashlib.sha256(hashlib.sha256(bytes.fromhex(modified_raw)).digest()).digest()
    return int.from_bytes(hash, 'big')

def check_sig(rs, neg):
    # 这个hash怎么来的呢？
    # 这个是把raw send tx中的 input[0] script去掉，取而代之 input[0] script中的 p2sh_redeem_script 而得到一个新的tx
    # 然后hash得到的
    hash = 0x981ace47e8d869bf17fd93558d536013841bebd38ed59270ad715174e8ab3e2d
    assert hash == get_sig_hash()
    assert rs < 2**(256*2)
    r = rs >> 256
    s = rs % 2**256

    assert 1 <= r < secp256k1_n
    assert 1 <= s < secp256k1_n

    z = hash

    x = r
    x2 = (x ** 3 + 7) % secp256k1_p
    assert secp256k1_p % 4 == 3
    y = pow(x2, (secp256k1_p+1)//4, secp256k1_p)
    assert y * y % secp256k1_p == x2

    if neg:
        y = secp256k1_p - y # FIXME

    R = (x, y)
    e = (secp256k1_n*2 - z) % secp256k1_n

    r_inv = pow(r, -1, secp256k1_n)
    assert r * r_inv % secp256k1_n == 1

    t1 = s * r_inv % secp256k1_n
    t2 = e * r_inv % secp256k1_n
    public_point = secp256k1_add(secp256k1_mul(t1, R), secp256k1_mul(t2, (secp256k1_Gx, secp256k1_Gy)))
    public_key = b'\x04' + public_point[0].to_bytes(32, 'big') + public_point[1].to_bytes(32, 'big')
    public_key = int.from_bytes(public_key, 'big')
    assert public_key == pub_key_3 or public_key == pub_key_2 or public_key == pub_key_1

check_sig(sig_1_parsed, True)
check_sig(sig_2_parsed, True)
